# Multiprocessing: scheduling
_COSC 208, Introduction to Computer Systems, 2023-04-21_

## Announcements
* Project 5 due Thursday, May 4

## Outline
* Warm-up
* Scheduling processes
* First In First Out (FIFO) scheduling
* Shortest Job First (SJF) scheduling
* Preemption
* Shortest Time to Completion First (STCF) scheduling

## Warm-up

* Q1: _What are all possible outputs of this program assuming no errors occur?_

In [ ]:
#include <stdio.h>
#include <unistd.h>
int main() {
    int pid = fork();
    if (pid < 0) {
        printf("Fork failed\n");
        return 1;
    } else if (pid == 0) {
        printf("Child\n");
    } else {
        wait(NULL);
        printf("Parent\n");
    }
    return 0;
}

Child
Parent


<p style="height:5em;"></p>

* Q2: _What are all possible outputs of this program, assuming no errors occur and the new process has PID 13346?_

In [ ]:
#include <stdio.h>
#include <unistd.h>
int main() {
    int pid = fork();
    printf("A %d\n", pid);
    if (pid < 0) {
        printf("Fork failed\n");
        return 1;
    } else if (pid == 0) {
        printf("B\n");
    } else {
        wait(NULL);
        printf("C\n");
    }
}

```
A 0
A 13346
B
C
```
OR
```
A 13346
A 0
B
C
```
OR
```
A 0
B
A 13346
C
```

<div style="page-break-after:always;"></div>

## Scheduling processes

* OS decides which process to run and for how long
* _What factors should the OS consider when making these decisions?_
    * Time for process to complete
    * Overhead of context switching
    * Fairness
    * User interaction
* For now, consider one of these metrics: turnaround time
    * T<sub>turnaround</sub> = T<sub>complete</sub> - T<sub>arrive</sub>
* For now, assume a process starts and runs to completion—i.e., no I/O and no preemption

## First In First Out (FIFO) scheduling

| Process | Arrival time | Duration | 
|---------|--------------|----------|
| A       | 0            | 15       |
| B       | 5            | 15       |
| C       | 10           | 15       |

* _What is the average turnaround time for the processes using FIFO?_
    ```
    ┌──────────────┬──────────────┬──────────────┐
    │ A            │ B            │ C            │
    └──────────────┴──────────────┴──────────────┘
    0              15             30             45
    ```
    * Average Turnaround = (15 + 25 + 35) / 3 = 25
* _What happens if C's duration is 60?_
    ```
    ┌──────────────┬──────────────┬────────────────────────────────────────────────────────┐
    │ A            │ B            │ C                                                      │
    └──────────────┴──────────────┴────────────────────────────────────────────────────────┘
    0              15             30                                                       90
    ```
    Average Turnaround = (15 + 25 + 80) / 3 = 40
* _What happens if A’s duration is 60?_
    ```
    ┌────────────────────────────────────────────────────────┬──────────────┬──────────────┐
    │ A                                                      │ B            │ C            │
    └────────────────────────────────────────────────────────┴──────────────┴──────────────┘
    0                                                        60             75             90
    ```
    * Average Turnaround = (60 + 70 + 80) / 3 = 70
* _How can we change the scheduling algorithm so the average turnaround time when A’s duration is 60 is more like the average turnaround time when C’s duration is 60?_

## Shortest Job First (SJF) scheduling

| Process | Arrival time | Duration | 
|---------|--------------|----------|
| A       | 0            | 60       |
| B       | 0            | 15       |
| C       | 0            | 15       |

* _What is the average turnaround time for the above processes using SJF?_
    ```
    ┌──────────────┬──────────────┬────────────────────────────────────────────────────────┐
    │ B            │ C            │ A                                                      │
    └──────────────┴──────────────┴────────────────────────────────────────────────────────┘
    0              15             30                                                       90
    ```
    * Average Turnaround = (15 + 30 + 90) / 3 = 45

| Process | Arrival time | Duration | 
|---------|--------------|----------|
| A       | 0            | 60       |
| B       | 5            | 15       |
| C       | 10           | 15       |

* _What is the average turnaround time for the above processes using SJF?_
    ```
    ┌────────────────────────────────────────────────────────┬──────────────┬──────────────┐
    │ A                                                      │ B            │ C            │
    └────────────────────────────────────────────────────────┴──────────────┴──────────────┘
    0                                                        60             75             90
    ```
    * Average Turnaround = (60 + 70 + 80) / 3 = 70
    * We're back to FIFO—_What happened!? How can we fix this?_

## Preemption

* OS only regains control when a system call occurs—e.g., read/write file, yield
    * Syscalls may occur infrequently, or never, due to program design, bugs, or malicious behavior
* _How does an OS forcibly regain control?_ — set a timer that raises an interrupt
    * Interrupt causes a trap instruction to be executed
    * Interrupts can also be raised by devices—e.g., Network Interface Card (NIC)
* _What must the OS do if it decides to run another process?_ — perform a context switch
    * Save the machine state associated with the process that was running—in particular, the contents of all registers are saved in the process's control structure
    * Restore the machine state associated with the process that should run—again, the contents of all registers are loaded from the process's control structure

## Shortest Time-to-Completion First (STCF) scheduling

* Allow preemption
* If a process arrives that has less computation remaining than the currently running process, then preempt the current process and run the new process
* Also known as Preemptive Shortest Job First (PSJF)

| Process | Arrival time | Duration | 
|---------|--------------|----------|
| A       | 0            | 60       |
| B       | 5            | 15       |
| C       | 10           | 15       |

* _What is the average turnaround time for the above processes using STCF scheduling?_
    ```
    ┌────┬──────────────┬──────────────┬───────────────────────────────────────────────────┐
    │ A  │ B            │ C            │ A                                                 │
    └────┴──────────────┴──────────────┴───────────────────────────────────────────────────┘
    0    5              20             35                                                   90
    ```
    * Average Turnaround = (90 + 15 + 25) / 3 = 43.3
* Now consider wait time: T<sub>wait</sub> = ∑ (T<sub>start_run</sub> - T<sub>become_ready</sub>)
    * A is waiting from time 5 to 35, so T<sub>wait</sub> = 30
    * B does not wait, so T<sub>wait</sub> = 0
    * C is waiting from time 10 to 20, so T<sub>wait</sub> = 10
    * Average Wait = (30 + 0 + 10) / 3 = 13.3

## Extra practice

* Q3: _What is the output produced by running `./progA`, assuming no errors occur?_

In [ ]:
// progA
#include <stdio.h>
#include <unistd.h>
int main() {
    int a = fork();
    if (a < 0) {
        perror("Fork failed");
        return 1;
    }
    else if (a == 0) {
        char *cmd[] = { "./progB", NULL };
        execv(cmd[0], cmd);
        printf("A 2nd gen\n");
        return 0;
    }
    else {
        wait(NULL);
        printf("A 1st gen\n");
        return 0;
    }
}

In [ ]:
// progB
#include <stdio.h>
#include <unistd.h>
int main() {
    int b = fork();
    if (b < 0) {
        perror("Fork failed");
        return 1;
    }
    else if (b == 0) {
        printf("B 2nd gen\n");
        return 0;
    }
    else {
        wait(NULL);
        printf("B 1st gen\n");
        return 0;
    }
}

```
B 2nd gen
B 1st gen
A 1st gen
```

<div style="page-break-after:always;"></div>

* Q4: _What does the following code output assuming no errors occur?_

In [ ]:
#include <stdio.h>
#include <unistd.h>
int main() {
    int x = 10;
    int y = 20;
    int retval = fork();
    if (retval == 0) {
        y -= 5;
    } 
    else if (retval > 0) {
        y+= 5;
    }
    printf("x=%d y=%d\n", x, y);
    return 0;
}

```
x=10 y=25
x=10 y=15
```
OR
```
x=10 y=15
x=10 y=25
```